# Exercices 4

[Télécharger l'exercice](../04_exercice.zip)

L'objectifs des exercices est de comprendre l'équation de diffusion en 1D, et sa discrétisation, d'implémenter un modèle simple de diffusion en 1D pour un problème de concentration, et de l'appliquer à un problème concret.

# Tutorial 1: Se familiariser avec l'approximation d'un dérivée spatiale

Pour résoudre un modèle (p.e. de diffusion), il nous faudra trés souvent dans ces modèles appoximer des dérivées, c.a.d. de facçon numérique (pas de façon exacte/analytique). Pour illustrer cela, nous définisons un vecteur $F$ avec la fonction sinus, sa dérivée "exacte" définit avec la fonction cosinus, et sa dérivée "approchée" défini au moyen d'une différence finie (approximation que nous utiliserons à mainte reprise dans ce cours). Le code suivant fait tout cela, et montre que la dérivée approximative est trés proche de la dérivée exacte:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Define x and dx
nx = 100
x = np.linspace(0, 2 * np.pi, nx)  # Example range of x
dx = x[1] - x[0]  # Compute dx

# Compute F, dFdx_exact, and dFdx_appro
F = np.sin(x)
dFdx_exact = np.cos(x)
dFdx_appro = (F[1:] - F[:-1]) / dx

# Compute midpoints (xm)
xm = (x[1:] + x[:-1]) / 2

# Plotting
plt.plot(x, dFdx_exact, label="dF/dx exact (blue)", color="blue")
plt.plot(xm, dFdx_appro, label="dF/dx approx (red)", color="red", linestyle="--")

# Add labels, legend, and title
plt.xlabel("x")
plt.ylabel("dF/dx")
plt.title("Comparaison entre dérivées Exact et Approximative")
plt.legend()

# Display the plot
plt.grid(True)
plt.show()


Notons que pour afficher les deux `dFdx_exact` et `dFdx_approx` sur un meme graphique, il a fallue calculer les points au centre des cellules `xm = (x[1:] + x[:-1]) / 2` cat la dérivée approximative est de taille `nx-1` puisque l'on perd une cellule lors de la différence finie `(F[1:] - F[:-1]) / dx`, il faut donc "plotter" `dFdx_approx` par rapport à `xm` et non `x` qui est de taille `nx`. 

# Exercice 1 : Diffusion simple d'une concentration

Vous allez maintenant rédiger un code qui permette de résoudre numériquement l'équation de diffusion de la concentration $C$:

$$ q = -D \frac{\partial C}{dx}, \qquad (1) $$
$$ \frac{\partial C}{\partial t} = -\frac{\partial q}{\partial x}. \qquad (2) $$

dans un domaine $[0,L_x]$, où $L_x=30$. 

Pour pouvoir faire le calcul, il faut également une condition initiale (concentration `C=C_i` dans la moitié gauche et `C=0` dans la moitié droite) et des conditions aux limites qui sont fixes à `x=0` et `x=Lx` (c.a.d. les mêmes que les conditions initiales). La figure suivante montre le résultat attendue aprés 20 secondes de diffusion.
 
![](./fig/DiffMod.png)

*Résultat de la diffusion après 20 secondes.*

 
Toutes les valeurs nécessaires sont dans la table suivante:

| **Paramètres**                      | **Valeurs**   |
|-------------------------------------|---------------|
| Longueur du modèle \(Lx\)           | 30 m          |
| Diffusivité \(D\)                   | 0.8 m²/s      |
| Condition initiale moitié gauche    | 500 ppm       |
| Condition initiale moitié droite    | 0 ppm         |
| Condition limite gauche             | 500 ppm       |
| Condition limite droite             | 0 ppm         |
| Temps total                         | 20 s          |

*Tableau 1 : Paramètres pour l'équation de diffusion.*

Pour résoudre l'exercice, veuillez utiliser la structure de code suivante pour l'initialisation et la boucle temporelle. Complétez ensuite les paramètres, l'initialisation et l'implémentation des équations (1) et (2).

```python
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import clear_output, display
 
# Physical parameters
D = ...     # diffusivite
Lx = ...    # longueur of model
C_l = ...   # Condition de bord gauche
C_r = ...   # Condition de bord droit
ttot = ...  # temps total

# Numerical parameters
nx = 200                      # nombre de cellules
dx = Lx / (nx - 1)            # taille d'une cellule
x = np.linspace(0, Lx, nx)    # coordonees des point du maillage
dt = dx**2 / (2.1 * D)        # condition sur le pas de temps
nt = ...                      # nombre de pas de temps
nout = 100                    # afficher la figure chaque 100e step

# Initial conditions
C = ...
C[:int(nx/2)] = ...

# create the figure outside the loop 
fig, ax = plt.subplots()

# loop
for it in range(nt):
    # Mettre a jour le flux q en fonction de C 
    q = ...
    # Mettre a jour dCdt
    dCdt = ...
    # Mettre a jour C
    C[1:-1] += ...

    # plot
    if it % nout == 0:
        clear_output(wait=True) 
        ax.cla() 
        ax.plot(x, C)
        ax.set_xlabel('Distance')
        ax.set_ylabel('Concentration')
        display(fig)
        plt.pause(0.1) 
```
### ✅ **À vous de faire !** 

# Exercice 2 : Modélisation de la fuite chimique de Daillens en avril 2015

Imaginez la fuite d'acide sulfurique qui a suivi l'accident ferroviaire de Daillens, VD, en 2015 près de la Venoge (voir [RTS](https://www.rts.ch/info/regions/6851235-en-cas-d-accident-un-wagon-de-chlore-pourrait-intoxiquer-20-000-personnes.html).  Les mesures lors de l’intervention et du rétablissement ont prévenus des contaminations de la nappe phréatique et de la Venoge. Le modèle représente une ligne qui va de la source de pollution (les wagons renversés qui ont déversé 25 tonnes de H<sub>2</sub>SO<sub>4</sub>) au puits local, la Venoge. 

![](./fig/fuite_chimique.png)

Il s'agit ici d'implémenter un modèle qui représente une version simplifiée de l'évolution de la pollution sans intervention post-accident (retrait de 1350 tonnes de sol). De plus, il sera considéré que le H<sub>2</sub>SO<sub>4</sub> ne se dégrade pas et ne se déplace que par diffusion. Reprenez le code de l'exercice précédent en adaptant la configuration initiale (Figure ci-dessous) et les paramètres de la table ci-dessous (en prenant garde aux unités). De plus, il faudra apporter de nouvelles choses au code :

- Vous effectuerez un changement des conditions aux frontières. Jusqu'à présent, les conditions aux frontières imposaient implicitement une concentration inchangée au bord, c'est-à-dire les mêmes que les conditions initiales étant donné que les valeurs de la solution aux bords ne sont pas mises à jour. Ici, la condition de bord $C = 0$ est une condition dite de "Dirichlet". Une autre condition de bord (dite de “Neumann”) impose $\frac{\partial C}{\partial x} = 0$, c'est-à-dire qu'il n'y a plus d'échange vers l'extérieur, en forçant un flux nul aux bords (en raison de l'équation (2)). Numériquement parlant, cela revient à imposer que les deux cellules de bord aient la même valeur. Dans le cas présent, on suppose une molasse imperméable qui rend tout flux vers l'extérieur impossible à l'extrémité gauche, et que la concentration à l'extrémité droite du modèle doit être maintenue à zéro afin que tout le polluant puisse quitter le domaine. On implémente ces deux conditions ainsi en Python dans la boucle temporelle après la mise à jour de la diffusion :

```python
C[0]  = C[1] 
C[-1] = 0
```

- Vous implémenterez une clause conditionnelle dans la boucle temporelle pour déterminer quand le niveau de contamination de la Venoge en H<sub>2</sub>SO<sub>4</sub> dépasse 0.1 ppm pour la première fois (utilisez un flag). Vous ajouterez un affichage de cette valeur à la fin du code.

- Vous implémenterez une clause conditionnelle dans la boucle temporelle pour déterminer quand le flux qui s'épanche dans la Venoge commence à diminuer pour la première fois (utilisez un flag). Vous ajouterez un affichage de cette valeur à la fin du code. Pour cela, il est utile de sauvegarder le flux avant la mise à jour. Cependant, la commande `q_old = q` ne sera pas efficace en Python, car `q_old = q` changera automatiquement lorsque `q` sera mis à jour (une spécificité de Python). En Python, il faut créer une "copie" explicitement avec la commande `q_old = np.copy(q)`, qui permettra de créer une nouvelle variable `q_old` retenant l'ancienne valeur de `q`, même après que celle-ci soit mise à jour.

Enfin, une fois le code terminé, faites des tests numériques avec plusieurs valeurs de $n_x$, et trouvez à partir de quelle valeur de $n_x$ les résultats ne dépendent plus sensiblement de ce paramètre numérique. 
 
![](./fig/fuite_chimique_shema_s5.png)

*Condition initiale pour l'exercice.*
 

| **Paramètres**                  | **Valeurs**         |
|---------------------------------|---------------------|
| Longueur, \(L_x\)               | 300 m               |
| Diffusivité, \(D\)              | $10^{-3}$ m²/s      |
| Concentration initiale          | 2000 ppm            |
| Tolérance arrêt modèle          | 0.001               |
| Temps total                     | 200 jours           |

*Tableau : Valeurs des paramètres pour l'exercice.*

### ✅ **À vous de faire !** 